# Judith

Let's start with the first heroine - Judith. In order to explore how she was represented in painting, we will use Wikidata.

In [1]:
!pip install --quiet rdflib SPARQLWrapper pandas

In [20]:
import rdflib
import ssl
import json
import pprint
import pandas as pd
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import URIRef

Judith = URIRef("https://www.wikidata.org/wiki/Q28532552")
depicts_relation = URIRef('http://www.wikidata.org/prop/direct/P180')
item = URIRef("https://www.wikidata.org/wiki/Property:P1629")
creative_work = URIRef("https://www.wikidata.org/wiki/Q17537576")

ssl._create_default_https_context = ssl._create_unverified_context
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql_wd = SPARQLWrapper(wikidata_endpoint)
sparql_wd.setReturnFormat(JSON)

Let's retrieve all records where Judith is an object

In [35]:
query = """
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX rdf: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?subject ?relation
    WHERE {
        ?subject ?relation wd:Q28532552 .
    }
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()
results = pd.DataFrame([
     {
         'subject': result['subject']['value'],
         'relation': result['relation']['value'],
     }
     for result in results['results']['bindings']
])

len(results)

290

Let's list all (unique) relations and find out whether all of them are informative for us and fit our needs

In [36]:
results['relation'].drop_duplicates().sort_values().reset_index(drop=True)

0                          http://schema.org/about
1         http://www.wikidata.org/prop/direct/P157
2         http://www.wikidata.org/prop/direct/P180
3        http://www.wikidata.org/prop/direct/P1889
4          http://www.wikidata.org/prop/direct/P26
5         http://www.wikidata.org/prop/direct/P301
6         http://www.wikidata.org/prop/direct/P674
7         http://www.wikidata.org/prop/direct/P921
8         http://www.wikidata.org/prop/direct/P971
9      http://www.wikidata.org/prop/statement/P157
10     http://www.wikidata.org/prop/statement/P180
11    http://www.wikidata.org/prop/statement/P1889
12      http://www.wikidata.org/prop/statement/P26
13     http://www.wikidata.org/prop/statement/P301
14     http://www.wikidata.org/prop/statement/P674
15     http://www.wikidata.org/prop/statement/P921
16     http://www.wikidata.org/prop/statement/P971
Name: relation, dtype: object

We see we have two versions (with 'direct' and 'statement' inside) of each relation, and both links redirect to the same page https://www.wikidata.org/wiki/Property:P[\d{3-4}].

We filter out following relations:

* http://schema.org/about: because it's just an information about the biblical character in different languages, not the art works.
* http://www.wikidata.org/prop/direct/P921 and http://www.wikidata.org/prop/statement/P921: because it's primary topic of a work, but not necessarily pictures
* http://www.wikidata.org/prop/direct/P1889 and http://www.wikidata.org/prop/statement/P1889: because it's about item that is different from another item. This is not necessarily about paintings, so we must remove it.
* http://www.wikidata.org/prop/direct/P674 and http://www.wikidata.org/prop/statement/P674: because it means characters which appear in this item (like plays, operas, operettas, books, comics, films, TV series, video games). We only consider paintings, so we don't need them.
* http://www.wikidata.org/prop/direct/P157 and http://www.wikidata.org/prop/statement/P157: because it's 'killed by'. We don't need it.
* http://www.wikidata.org/prop/direct/P301 and http://www.wikidata.org/prop/statement/P301: because it's a 'category'.
* http://www.wikidata.org/prop/direct/P971 and http://www.wikidata.org/prop/statement/P971: because it's'category combines topics', it's not about painting.
* http://www.wikidata.org/prop/direct/P26 and http://www.wikidata.org/prop/statement/P26: because it's 'spouse'. This is also not about art works.

Hence the only relation that we are interested in is 'depicts' (http://www.wikidata.org/prop/direct/P180 + http://www.wikidata.org/prop/statement/P180):

In [37]:
mask = results['relation'].isin([
  'http://www.wikidata.org/prop/direct/P180',
  'http://www.wikidata.org/prop/statement/P180',
])

results = results[mask].reset_index(drop=True)

len(results)

242


Let's inspect the subjects:

In [38]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    print(results['subject'].drop_duplicates())

0                                                      http://www.wikidata.org/entity/Q19912353
1                                                      http://www.wikidata.org/entity/Q19912741
2                                                      http://www.wikidata.org/entity/Q19925957
3                                                      http://www.wikidata.org/entity/Q19947516
4                                                      http://www.wikidata.org/entity/Q20188059
5                                                      http://www.wikidata.org/entity/Q20198819
6                                                      http://www.wikidata.org/entity/Q20268690
7                                                      http://www.wikidata.org/entity/Q20271630
8                                                      http://www.wikidata.org/entity/Q20277944
9                                                      http://www.wikidata.org/entity/Q20441065
10                                      

We notice some unusual links which are called statements in wikidata. The statement links allow to jump immediately to the statement label that is indicated, but we don't need them because these are links to the pages with pictures we already have. We should exclude them.

In [39]:
mask = ~results['subject'].str.contains('statement')
results = results[mask].reset_index(drop=True)

len(results)

121

Now we notice that only one variant (with 'direct' inside) of 'depicts' relation is left in our dataset.

In [40]:
results['relation'].unique()

array(['http://www.wikidata.org/prop/direct/P180'], dtype=object)

So, we could actually use this particular relation to obtain the same set of artworks:
```
WHERE {
    ?subject wdt:P180 wd:Q28532552 .
}
```


We also notice that we have а group of paintings by and after Cristofano Allori (http://www.wikidata.org/entity/Q43150796). It consists of six works. Five of them were in results of our query, but one (https://www.wikidata.org/wiki/Q27978379) was not. We should delete the link to the group of paintings and will later add the link to the painting not included in the dataset.

In order to find out in which cultures the plot is most popular, we need more infomration. We will gather more information from wikidata about the paintings and also look at the authors of the paintings.

# JUDITH ARTWORKS

In [41]:
query = """
      SELECT ?uri ?name ?image_uri
             (YEAR(?inception_date) AS ?inception_year) ?inception_precision
             (YEAR(?earliest_date) AS ?earliest_year) (YEAR(?latest_date) AS ?latest_year)
             ?location_uri ?collection_uri ?collection_name ?creator_uri
      WHERE {
          ?uri wdt:P31 wd:Q3305213 .
          ?uri wdt:P180 wd:Q28532552 .
          OPTIONAL { ?uri wdt:P18 ?image_uri . }
          OPTIONAL { ?uri p:P571/pq:P1319 ?earliest_date . }
          OPTIONAL { ?uri p:P571/pq:P1326 ?latest_date . }
          OPTIONAL {
              ?uri p:P571/psv:P571 [
                  wikibase:timeValue ?inception_date ;
                  wikibase:timePrecision ?inception_precision ;
              ]
          }
          OPTIONAL { ?uri wdt:P276 ?location_uri . }
          OPTIONAL { ?uri wdt:P195 ?collection_uri . }
          OPTIONAL { ?uri wdt:P170 ?creator_uri . }
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en,de" .
            ?uri rdfs:label ?name .
            ?collection_uri rdfs:label ?collection_name .
          }
      }
 """

sparql_wd.setQuery(query)
artworks_response = sparql_wd.query().convert()

keys = [
    'uri', 'name', 'image_uri', 'inception_year', 'inception_precision',
    'earliest_year', 'latest_year', 'location_uri', 'collection_uri',
    'collection_name', 'creator_uri'
]
artworks_response = [
    {key: result.get(key, {}).get('value', None) for key in keys}
    for result in artworks_response['results']['bindings']
]

pprint(artworks_response[0])
pprint(artworks_response[1])
len(artworks_response)

{'collection_name': 'Statens Museum for Kunst',
 'collection_uri': 'http://www.wikidata.org/entity/Q671384',
 'creator_uri': 'http://www.wikidata.org/entity/Q544264',
 'earliest_year': None,
 'image_uri': 'http://commons.wikimedia.org/wiki/Special:FilePath/Denys%20Calvaert%20-%20Judith%20with%20the%20Head%20of%20Holofernes%20-%20KMS981%20-%20Statens%20Museum%20for%20Kunst.jpg',
 'inception_precision': None,
 'inception_year': None,
 'latest_year': None,
 'location_uri': 'http://www.wikidata.org/entity/Q671384',
 'name': 'Judith with the Head of Holofernes',
 'uri': 'http://www.wikidata.org/entity/Q20277944'}
{'collection_name': 'Museum of Fine Arts',
 'collection_uri': 'http://www.wikidata.org/entity/Q49133',
 'creator_uri': 'http://www.wikidata.org/entity/Q982384',
 'earliest_year': None,
 'image_uri': 'http://commons.wikimedia.org/wiki/Special:FilePath/Massimo%20Stanzione%20-%20Judith%20with%20the%20Head%20of%20Holofernes%20-%2094.181%20-%20Museum%20of%20Fine%20Arts.jpg',
 'inception

152

# JUDITH CREATORS

In [42]:
creator_uris = set()

for painting in artworks_response:
  if painting['creator_uri'] and 'well-known' not in painting['creator_uri']:
    creator_uris.add(painting['creator_uri'])


creator_uris = [f"wd:{uri[uri.rindex('/') + 1:]}" for uri in creator_uris]

print(creator_uris[0])
print(creator_uris[1])
len(creator_uris)

wd:Q317920
wd:Q938607


78

In [43]:
query = """
    SELECT ?uri ?name ?gender ?citizenship
        (YEAR(?birth_date) AS ?birth_year) (YEAR(?death_date) AS ?death_year)
        ?birth_place_uri ?death_place_uri ?work_location_uri
    WHERE {
        VALUES ?uri {""" +  ' '.join(creator_uris)  + """}
        OPTIONAL { ?uri wdt:P21 ?gender_uri . }
        OPTIONAL { ?uri wdt:P27 ?citizenship . }
        OPTIONAL { ?uri wdt:P569 ?birth_date . }
        OPTIONAL { ?uri wdt:P570 ?death_date . }
        OPTIONAL { ?uri wdt:P19 ?birth_place_uri . }
        OPTIONAL { ?uri wdt:P20 ?death_place_uri . }
        OPTIONAL { ?uri wdt:P937 ?work_location_uri . }
        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en,de" .
            ?uri rdfs:label ?name .
            ?gender_uri rdfs:label ?gender .
        }
    }
 """

sparql_wd.setQuery(query)
creators_response = sparql_wd.query().convert()

keys = [
    'uri', 'name', 'gender', 'citizenship', 'birth_year', 'death_year',
    'birth_place_uri', 'death_place_uri', 'work_location_uri'
]
creators_response = [
    {key: result.get(key, {}).get('value', None) for key in keys}
    for result in creators_response['results']['bindings']
]

pprint(creators_response[0])
pprint(creators_response[1])
len(creators_response)

{'birth_place_uri': 'http://www.wikidata.org/entity/Q1372769',
 'birth_year': '1746',
 'citizenship': 'http://www.wikidata.org/entity/Q29',
 'death_place_uri': 'http://www.wikidata.org/entity/Q1479',
 'death_year': '1828',
 'gender': 'male',
 'name': 'Francisco de Goya',
 'uri': 'http://www.wikidata.org/entity/Q5432',
 'work_location_uri': 'http://www.wikidata.org/entity/Q29'}
{'birth_place_uri': 'http://www.wikidata.org/entity/Q1372769',
 'birth_year': '1746',
 'citizenship': 'http://www.wikidata.org/entity/Q29',
 'death_place_uri': 'http://www.wikidata.org/entity/Q1479',
 'death_year': '1828',
 'gender': 'male',
 'name': 'Francisco de Goya',
 'uri': 'http://www.wikidata.org/entity/Q5432',
 'work_location_uri': 'http://www.wikidata.org/entity/Q38'}


392

# JUDITH LOCATIONS

In [44]:
location_uris = set()

for creator in creators_response:
    if creator['birth_place_uri']:
        location_uris.add(creator['birth_place_uri'])
    if creator['death_place_uri']:
        location_uris.add(creator['death_place_uri'])
    if creator['work_location_uri']:
        location_uris.add(creator['work_location_uri'])

for artwork in artworks_response:
    if artwork['location_uri']:
        location_uris.add(artwork['location_uri'])

location_uris = [f"wd:{uri[uri.rindex('/') + 1:]}" for uri in location_uris]

print(location_uris[0])
print(location_uris[1])
len(location_uris)

wd:Q7194793
wd:Q865736


186

In [45]:
query = """
    SELECT ?uri ?name ?lat ?lon
    WHERE {
        VALUES ?uri {""" +  ' '.join(location_uris)  + """}
        ?uri p:P625/psv:P625 [ wikibase:geoLatitude ?lat ; wikibase:geoLongitude ?lon ; ]
        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en,de" .
            ?uri rdfs:label ?name .
        }
    }
 """

sparql_wd.setQuery(query)
locations_response = sparql_wd.query().convert()

keys = ['uri', 'name', 'lat', 'lon']
locations_response = [
    {key: result.get(key, {}).get('value', None) for key in keys}
    for result in locations_response['results']['bindings']
]

pprint(locations_response[0])
pprint(locations_response[1])
len(locations_response)

{'lat': '43.768438888889',
 'lon': '11.2559',
 'name': 'Uffizi',
 'uri': 'http://www.wikidata.org/entity/Q51252'}
{'lat': '40.2',
 'lon': '-3.5',
 'name': 'Spain',
 'uri': 'http://www.wikidata.org/entity/Q29'}


197

Create dicts from lists

In [46]:
artworks_by_uri = {item['uri']: item for item in artworks_response}
creators_by_uri = {item['uri']: item for item in creators_response}
locations_by_uri = {item['uri']: item for item in locations_response}

print(artworks_by_uri.keys())
print(creators_by_uri.keys())
print(locations_by_uri.keys())

dict_keys(['http://www.wikidata.org/entity/Q20277944', 'http://www.wikidata.org/entity/Q20549802', 'http://www.wikidata.org/entity/Q28001898', 'http://www.wikidata.org/entity/Q28002900', 'http://www.wikidata.org/entity/Q28071787', 'http://www.wikidata.org/entity/Q29909510', 'http://www.wikidata.org/entity/Q30057872', 'http://www.wikidata.org/entity/Q30097339', 'http://www.wikidata.org/entity/Q29938854', 'http://www.wikidata.org/entity/Q30096134', 'http://www.wikidata.org/entity/Q28778013', 'http://www.wikidata.org/entity/Q52207936', 'http://www.wikidata.org/entity/Q107650490', 'http://www.wikidata.org/entity/Q29479966', 'http://www.wikidata.org/entity/Q29919256', 'http://www.wikidata.org/entity/Q29938834', 'http://www.wikidata.org/entity/Q29938841', 'http://www.wikidata.org/entity/Q30098509', 'http://www.wikidata.org/entity/Q55237558', 'http://www.wikidata.org/entity/Q55853411', 'http://www.wikidata.org/entity/Q57728663', 'http://www.wikidata.org/entity/Q58331423', 'http://www.wikidata

Join data

In [47]:
artworks = list(artworks_by_uri.values()) # transform back to list
artworks = [obj.copy() for obj in artworks] # need to create copies because we will modify the objects

for artwork in artworks:
    artwork_creator = creators_by_uri.get(artwork['creator_uri'], {})
    for key in ['name', 'gender', 'citizenship']:
        artwork[f'creator_{key}'] = artwork_creator.get(key, None)

    artwork_location = locations_by_uri.get(artwork['location_uri'], {})
    for key in ['name', 'lat', 'lon']:
        artwork[f'location_{key}'] = artwork_location.get(key, None)

pd.DataFrame(artworks)

,uri,name,image_uri,inception_year,inception_precision,earliest_year,latest_year,location_uri,collection_uri,collection_name,creator_uri,creator_name,creator_gender,creator_citizenship,location_name,location_lat,location_lon
0,http://www.wikidata.org/entity/Q20277944,Judith with the Head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q671384,http://www.wikidata.org/entity/Q671384,Statens Museum for Kunst,http://www.wikidata.org/entity/Q544264,Denys Calvaert,male,http://www.wikidata.org/entity/Q6581823,Statens Museum for Kunst,55.68889,12.57861
1,http://www.wikidata.org/entity/Q20549802,Judith with the Head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q49133,http://www.wikidata.org/entity/Q49133,Museum of Fine Arts,http://www.wikidata.org/entity/Q982384,Massimo Stanzione,male,http://www.wikidata.org/entity/Q173065,Museum of Fine Arts,42.339166666667,-71.094166666667
2,http://www.wikidata.org/entity/Q28001898,Judith,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q303139,http://www.wikidata.org/entity/Q303139,Belvedere,http://www.wikidata.org/entity/Q28813418,Leopold Pollak,male,http://www.wikidata.org/entity/Q131964,Belvedere,48.19139,16.38049
3,http://www.wikidata.org/entity/Q28002900,Judith mit dem Haupt des Holofernes,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q303139,http://www.wikidata.org/entity/Q303139,Belvedere,http://www.wikidata.org/entity/Q28055102,Joseph Hasslwander,male,http://www.wikidata.org/entity/Q28513,Belvedere,48.19139,16.38049
4,http://www.wikidata.org/entity/Q28071787,Judith with the Head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,None,http://www.wikidata.org/entity/Q28065304,Collection Goudstikker heirs,http://www.wikidata.org/.well-known/genid/0e20...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,http://www.wikidata.org/entity/Q27700979,Judith offering the Head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,1636,9,None,None,http://www.wikidata.org/entity/Q160112,http://www.wikidata.org/entity/Q160112,Museo del Prado,http://www.wikidata.org/entity/Q541736,Salomon de Bray,male,http://www.wikidata.org/entity/Q170072,Museo del Prado,40.413888888889,-3.6922222222222
107,http://www.wikidata.org/entity/Q27954769,Judith,http://commons.wikimedia.org/wiki/Special:File...,1650,7,1640,1650,http://www.wikidata.org/entity/Q1848918,http://www.wikidata.org/entity/Q1848918,Museo Nacional de Bellas Artes,http://www.wikidata.org/entity/Q951884,Gerard Seghers,male,http://www.wikidata.org/entity/Q6581823,Museo Nacional de Bellas Artes,-34.58405556,-58.39280556
108,http://www.wikidata.org/entity/Q28027812,Judith with the Head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,1613,9,None,None,http://www.wikidata.org/entity/Q1434767,http://www.wikidata.org/entity/Q1459037,Royal Collection,http://www.wikidata.org/entity/Q352482,Cristofano Allori,male,http://www.wikidata.org/entity/Q154849,Queen's Gallery,51.4998,-0.142408
109,http://www.wikidata.org/entity/Q28100917,Judith with the head of Holofernes,http://commons.wikimedia.org/wiki/Special:File...,1525,9,None,None,http://www.wikidata.org/entity/Q7194793,http://www.wikidata.org/entity/Q7194793,Pinacoteca Querini Stampalia,http://www.wikidata.org/entity/Q1968947,Vincenzo Catena,male,http://www.wikidata.org/entity/Q4948,Pinacoteca Querini Stampalia,45.43642,12.34107


In [ ]:
pd.DataFrame(artworks).to_excel('./salome.xlsx')

# SALOME

Now let's move on to another heroine - Salome. We'll do the same thing we did for Judith.

In [48]:
query = """
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX rdf: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?subject ?relation
    WHERE {
        ?subject ?relation wd:Q228896 .
    }
"""
sparql_wd.setQuery(query)

results = sparql_wd.query().convert()
results = pd.DataFrame([
     {
         'subject': result['subject']['value'],
         'relation': result['relation']['value'],
     }
     for result in results['results']['bindings']
])

len(results)

423

In [49]:
results['relation'].drop_duplicates().sort_values().reset_index(drop=True)

0                          http://schema.org/about
1         http://www.wikidata.org/prop/direct/P138
2         http://www.wikidata.org/prop/direct/P180
3        http://www.wikidata.org/prop/direct/P1889
4          http://www.wikidata.org/prop/direct/P25
5          http://www.wikidata.org/prop/direct/P26
6         http://www.wikidata.org/prop/direct/P301
7        http://www.wikidata.org/prop/direct/P3342
8          http://www.wikidata.org/prop/direct/P40
9         http://www.wikidata.org/prop/direct/P674
10        http://www.wikidata.org/prop/direct/P921
11    http://www.wikidata.org/prop/reference/P3452
12     http://www.wikidata.org/prop/statement/P138
13     http://www.wikidata.org/prop/statement/P180
14    http://www.wikidata.org/prop/statement/P1889
15      http://www.wikidata.org/prop/statement/P25
16      http://www.wikidata.org/prop/statement/P26
17     http://www.wikidata.org/prop/statement/P301
18    http://www.wikidata.org/prop/statement/P3342
19      http://www.wikidata.org

In [50]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    print(results['subject'].drop_duplicates())

0                                                                                                            http://www.wikidata.org/entity/Q19925990
1                                                                                                            http://www.wikidata.org/entity/Q20026920
2                                                                                                            http://www.wikidata.org/entity/Q20026949
3                                                                                                            http://www.wikidata.org/entity/Q20054305
4                                                                                                            http://www.wikidata.org/entity/Q20054309
5                                                                                                            http://www.wikidata.org/entity/Q20054386
6                                                                                                   

In [51]:
query = """
      SELECT ?uri ?name ?image_uri
             (YEAR(?inception_date) AS ?inception_year) ?inception_precision
             (YEAR(?earliest_date) AS ?earliest_year) (YEAR(?latest_date) AS ?latest_year)
             ?location_uri ?collection_uri ?collection_name ?creator_uri
      WHERE {
          ?uri wdt:P31 wd:Q3305213 .
          ?uri wdt:P180 wd:Q228896 .
          OPTIONAL { ?uri wdt:P18 ?image_uri . }
          OPTIONAL { ?uri p:P571/pq:P1319 ?earliest_date . }
          OPTIONAL { ?uri p:P571/pq:P1326 ?latest_date . }
          OPTIONAL {
              ?uri p:P571/psv:P571 [
                  wikibase:timeValue ?inception_date ;
                  wikibase:timePrecision ?inception_precision ;
              ]
          }
          OPTIONAL { ?uri wdt:P276 ?location_uri . }
          OPTIONAL { ?uri wdt:P195 ?collection_uri . }
          OPTIONAL { ?uri wdt:P170 ?creator_uri . }
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en,de" .
            ?uri rdfs:label ?name .
            ?collection_uri rdfs:label ?collection_name .
          }
      }
 """

sparql_wd.setQuery(query)
artworks_response = sparql_wd.query().convert()

keys = [
    'uri', 'name', 'image_uri', 'inception_year', 'inception_precision',
    'earliest_year', 'latest_year', 'location_uri', 'collection_uri',
    'collection_name', 'creator_uri'
]
artworks_response = [
    {key: result.get(key, {}).get('value', None) for key in keys}
    for result in artworks_response['results']['bindings']
]

pprint(artworks_response[0])
pprint(artworks_response[1])
len(artworks_response)

{'collection_name': "Musée d'Orsay",
 'collection_uri': 'http://www.wikidata.org/entity/Q23402',
 'creator_uri': 'http://www.wikidata.org/entity/Q3187858',
 'earliest_year': None,
 'image_uri': 'http://commons.wikimedia.org/wiki/Special:FilePath/19%20Salome.JPG',
 'inception_precision': None,
 'inception_year': None,
 'latest_year': None,
 'location_uri': 'http://www.wikidata.org/entity/Q3075877',
 'name': 'Salomé',
 'uri': 'http://www.wikidata.org/entity/Q17495573'}
{'collection_name': 'Statens Museum for Kunst',
 'collection_uri': 'http://www.wikidata.org/entity/Q671384',
 'creator_uri': 'http://www.wikidata.org/entity/Q5599',
 'earliest_year': None,
 'image_uri': 'http://commons.wikimedia.org/wiki/Special:FilePath/Peter%20Paul%20Rubens%20-%20The%20Feast%20of%20Herod.%20Salome%20Bringing%20the%20Head%20of%20St%20John%20the%20Baptist%20on%20a%20Charger%20-%20KMSsp192%20-%20Statens%20Museum%20for%20Kunst.jpg',
 'inception_precision': None,
 'inception_year': None,
 'latest_year': None,

135

# SALOME CREATORS

In [52]:
creator_uris = set()

for painting in artworks_response:
  if painting['creator_uri'] and 'well-known' not in painting['creator_uri']:
    creator_uris.add(painting['creator_uri'])


creator_uris = [f"wd:{uri[uri.rindex('/') + 1:]}" for uri in creator_uris]

print(creator_uris[0])
print(creator_uris[1])
len(creator_uris)

wd:Q3187858
wd:Q381248


70

In [53]:
query = """
    SELECT ?uri ?name ?gender ?citizenship
        (YEAR(?birth_date) AS ?birth_year) (YEAR(?death_date) AS ?death_year)
        ?birth_place_uri ?death_place_uri ?work_location_uri
    WHERE {
        VALUES ?uri {""" +  ' '.join(creator_uris)  + """}
        OPTIONAL { ?uri wdt:P21 ?gender_uri . }
        OPTIONAL { ?uri wdt:P27 ?citizenship . }
        OPTIONAL { ?uri wdt:P569 ?birth_date . }
        OPTIONAL { ?uri wdt:P570 ?death_date . }
        OPTIONAL { ?uri wdt:P19 ?birth_place_uri . }
        OPTIONAL { ?uri wdt:P20 ?death_place_uri . }
        OPTIONAL { ?uri wdt:P937 ?work_location_uri . }
        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en,de" .
            ?uri rdfs:label ?name .
            ?gender_uri rdfs:label ?gender .
        }
    }
 """

sparql_wd.setQuery(query)
creators_response = sparql_wd.query().convert()

keys = [
    'uri', 'name', 'gender', 'citizenship', 'birth_year', 'death_year',
    'birth_place_uri', 'death_place_uri', 'work_location_uri'
]
creators_response = [
    {key: result.get(key, {}).get('value', None) for key in keys}
    for result in creators_response['results']['bindings']
]

pprint(creators_response[0])
pprint(creators_response[1])
len(creators_response)

{'birth_place_uri': 'http://www.wikidata.org/entity/Q3167',
 'birth_year': '1577',
 'citizenship': 'http://www.wikidata.org/entity/Q622783',
 'death_place_uri': 'http://www.wikidata.org/entity/Q12892',
 'death_year': '1640',
 'gender': 'male',
 'name': 'Peter Paul Rubens',
 'uri': 'http://www.wikidata.org/entity/Q5599',
 'work_location_uri': 'http://www.wikidata.org/entity/Q84'}
{'birth_place_uri': 'http://www.wikidata.org/entity/Q3167',
 'birth_year': '1577',
 'citizenship': 'http://www.wikidata.org/entity/Q622783',
 'death_place_uri': 'http://www.wikidata.org/entity/Q12892',
 'death_year': '1640',
 'gender': 'male',
 'name': 'Peter Paul Rubens',
 'uri': 'http://www.wikidata.org/entity/Q5599',
 'work_location_uri': 'http://www.wikidata.org/entity/Q90'}


297

# SALOME LOCATIONS

In [54]:
location_uris = set()

for creator in creators_response:
    if creator['birth_place_uri']:
        location_uris.add(creator['birth_place_uri'])
    if creator['death_place_uri']:
        location_uris.add(creator['death_place_uri'])
    if creator['work_location_uri']:
        location_uris.add(creator['work_location_uri'])

for artwork in artworks_response:
    if artwork['location_uri']:
        location_uris.add(artwork['location_uri'])

location_uris = [f"wd:{uri[uri.rindex('/') + 1:]}" for uri in location_uris]

print(location_uris[0])
print(location_uris[1])
len(location_uris)

wd:Q1794
wd:Q26421


206

In [56]:
Salome_artworks_by_uri = {item['uri']: item for item in artworks_response}
Salome_creators_by_uri = {item['uri']: item for item in creators_response}
Salome_locations_by_uri = {item['uri']: item for item in locations_response}


In [ ]:
Salome_artworks = list(Salome_artworks_by_uri.values()) # transform back to list
Salome_artworks = [obj.copy() for obj in Salome_artworks] # need to create copies because we will modify the objects

for artwork in Salome_artworks:
    Salome_artwork_creator = Salome_creators_by_uri.get(artwork['creator_uri'], {})
    for key in ['name', 'gender', 'citizenship']:
        artwork[f'creator_{key}'] = Salome_artwork_creator.get(key, None)

    Salome_artwork_location = Salome_locations_by_uri.get(artwork['location_uri'], {})
    for key in ['name', 'lat', 'lon']:
        artwork[f'location_{key}'] = Salome_artwork_location.get(key, None)

pd.DataFrame(Salome_artworks)

,uri,name,image_uri,inception_year,inception_precision,earliest_year,latest_year,location_uri,collection_uri,collection_name,creator_uri,creator_name,creator_gender,creator_citizenship,location_name,location_lat,location_lon
0,http://www.wikidata.org/entity/Q17495573,Salomé,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q3075877,http://www.wikidata.org/entity/Q23402,Musée d'Orsay,http://www.wikidata.org/entity/Q3187858,Juana Romani,female,http://www.wikidata.org/entity/Q172579,None,None,None
1,http://www.wikidata.org/entity/Q20422087,The Feast of Herod. Salome Bringing the Head o...,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,http://www.wikidata.org/entity/Q671384,http://www.wikidata.org/entity/Q671384,Statens Museum for Kunst,http://www.wikidata.org/entity/Q5599,Peter Paul Rubens,male,http://www.wikidata.org/entity/Q622783,Statens Museum for Kunst,55.68889,12.57861
2,http://www.wikidata.org/entity/Q88804762,Salome with the Head of John the Baptist,http://commons.wikimedia.org/wiki/Special:File...,None,None,None,None,None,http://www.wikidata.org/entity/Q3867551,Musei di Strada Nuova,http://www.wikidata.org/entity/Q949627,Matthias Stom,male,http://www.wikidata.org/entity/Q6581823,None,None,None
3,http://www.wikidata.org/entity/Q108732966,Q108732966,None,None,None,None,None,None,http://www.wikidata.org/entity/Q10333407,Museum Baron Martin,http://www.wikidata.org/entity/Q3048215,Edouard Frederic Wilhelm Richter,male,http://www.wikidata.org/entity/Q142,None,None,None
4,http://www.wikidata.org/entity/Q27980426,Salome with the head of John the Baptist,http://commons.wikimedia.org/wiki/Special:File...,1514,8,1512,1516,http://www.wikidata.org/entity/Q95569,http://www.wikidata.org/entity/Q95569,Kunsthistorisches Museum,http://www.wikidata.org/entity/Q1056977,Cesare da Sesto,male,None,Kunsthistorisches Museum,48.203611111111,16.361666666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,http://www.wikidata.org/entity/Q24436847,The Feast of Herod,http://commons.wikimedia.org/wiki/Special:File...,1531,9,None,None,http://www.wikidata.org/entity/Q403080,http://www.wikidata.org/entity/Q403080,Wadsworth Atheneum Museum of Art,http://www.wikidata.org/entity/Q191748,Lucas Cranach the Elder,male,http://www.wikidata.org/entity/Q12548,None,None,None
99,http://www.wikidata.org/entity/Q26831225,Salome,http://commons.wikimedia.org/wiki/Special:File...,1515,9,None,None,http://www.wikidata.org/entity/Q1752085,http://www.wikidata.org/entity/Q1752085,Norton Simon Museum,http://www.wikidata.org/entity/Q47551,Titian,male,http://www.wikidata.org/entity/Q4948,None,None,None
100,http://www.wikidata.org/entity/Q26831241,Salome,http://commons.wikimedia.org/wiki/Special:File...,1550,9,None,None,http://www.wikidata.org/entity/Q160112,http://www.wikidata.org/entity/Q160112,Museo del Prado,http://www.wikidata.org/entity/Q47551,Titian,male,http://www.wikidata.org/entity/Q4948,Museo del Prado,40.413888888889,-3.6922222222222
101,http://www.wikidata.org/entity/Q27766932,Triptych: ''Herodias receives the head of John...,None,1550,8,None,None,http://www.wikidata.org/entity/Q768717,http://www.wikidata.org/.well-known/genid/f881...,http://www.wikidata.org/.well-known/genid/f881...,http://www.wikidata.org/entity/Q320980,Lucas van Leyden,male,http://www.wikidata.org/entity/Q27996474,None,None,None


In [ ]:
pd.DataFrame(Salome_artworks).to_excel('./salome.xlsx')